Das Ziel: die gescrapten Daten, die in Form von über 140'000 html-Files vorliegen in ein Dataframe zu überführen, die als CSV-Datei gespeichert wird. Erst dann lassen sich mit den Daten erste Analysen machen. 

Hier haben wir die Parsing-Sequenz, die wir im Notebook "MedReg Parsing Explore" erstellt haben. Als erstes überführen wir die Daten, die wir mit BeautifulSoup rausholen in ein Minidict. 

In [21]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
import datetime
from tqdm import tqdm

In [22]:
def get_gender(soup):
    try:
        img_source = soup.find('h3').find("img")["src"]
        if "venus" in img_source:
            return "w"
        else:
            return "m"
    except:
        return "unbekannt"
    #durch Barny mit try/except ergänzt, für den Fall leerer Einträge. 
    

In [23]:
path = 'MedReg_Scraped_Data_20K/'
file_list = os.listdir(path)

file = open(path + file_list[4805], 'r')
text = file.read()
soup = BeautifulSoup(text, 'html.parser')


name = soup.find('h3').text.replace('\xa0','')
nationalität = soup.find('p').text
geschlecht = get_gender(soup)
adresse = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
beruf = soup.find('tbody').find('td').text
jahr_diplom = soup.find('tbody').find('td').find_next('td').text
land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td').text
datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td').text
weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').text
jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').text
land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td').text

minidict = {'Name': name,
           'Adresse': adresse,
            'Geschlecht': geschlecht,
            'Beruf': beruf,
            'Jahr Diplom': jahr_diplom,
            'Land Diplom': land_diplom,
            'Anerkennungsdatum': datum_anerkennung,
            'Weiterbildung': weiterbildungstitel,
            'Jahr Weiterbildung': jahr_wbtitel,
            'Land Weiterbildung': land_wbtitel,
            'Nation': nationalität
           }

#Umwandlung des Anerkennungsdatum später dann mit der Formel:
#date_time_str = re.findall(r'\d+.\d+.\d+', datum_anerkennung)[0]
#date_time_obj = datetime.datetime.strptime(date_time_str, '%d.%m.%Y')
#date_time_obj.strftime("%d.%m.%Y")

In [24]:
pd.DataFrame([minidict])

,Name,Adresse,Geschlecht,Beruf,Jahr Diplom,Land Diplom,Anerkennungsdatum,Weiterbildung,Jahr Weiterbildung,Land Weiterbildung,Nation
0,"Blaschegg-Honsalek, Irma","Dufourstr. 169,8008 Zürich,ZH",w,Ärztin/Arzt,1969,Deutschland,Anerkennung \n (13.08.2002)...,Psychiatrie und Psychotherapie,2002,Schweiz,Nationalität: Deutschland (DE)Sprachkenntnisse...


In [25]:
soup.find('p')

<p>Nationalität: Deutschland (DE)<br/>Sprachkenntnisse: Deutsch<br/>GLN: 7601000677815<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) "> </a><br/>UID: CHE103519026<a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gründen identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge für die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen geführt."> </a></p>

In [47]:
soup.find('p').text

'Nationalität: Deutschland (DE)Sprachkenntnisse: DeutschGLN: 7601000677815\xa0UID: CHE103519026\xa0'

Das weitere Aufstückeln wird kompliziert. Lieber später in Pandas. 
https://stackoverflow.com/questions/48216573/converting-br-to-p-p-in-python?noredirect=1&lq=1

Ergebnis aus Projektvorstellung 10.12.2019: Dieses Problem erst später lösen. Prinzip: lieber dreckige Daten, als keine Daten oder zu komplizierte Formeln, die bei Ausnahmen Fehler produzieren. 

## Frühere Versuche

Ok. Sieht für einen Eintrag gut aus. Jetzt müssen wir das noch für alle Files in Serie machen. Dazu verpacken wir den obigen Datenextrakt-Code in eine Funktion. 

In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
import datetime
from tqdm import tqdm

#Funktion, um aus dem Logo einen Eintrag für das Geschlecht zu generieren
def get_gender(soup):
    img_source = soup.find('h3').find("img")["src"]
    if "venus" in img_source:
        return "w"
    else:
        return "m"

#Diese Funktion zieht die Daten aus den einzelnen html-Einträgen
def extract(soup):
    
    medreg_auszug = []
    
    for element in soup: #Was passiert mit element?    
        
        name = soup.find('h3').text.replace('\xa0','')
        #nationalität = soup.find('p')
        geschlecht = get_gender(soup)
        adresse = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
        beruf = soup.find('tbody').find('td').text
        jahr_diplom = soup.find('tbody').find('td').find_next('td').text
        land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td').text
        datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td').text
        weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').text
        jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').text
        land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td').text

        minidict = {'Name': name,
                   'Adresse': adresse,
                    'Geschlecht': geschlecht,
                    'Beruf': beruf,
                    'Jahr Diplom': jahr_diplom,
                    'Land Diplom': land_diplom,
                    'Anerkennungsdatum': datum_anerkennung,
                    'Weiterbildung': weiterbildungstitel,
                    'Jahr Weiterbildung': jahr_wbtitel,
                    'Land Weiterbildung': land_wbtitel
                   }

        medreg_auszug.append(minidict)
    
    return medreg_auszug


Die Funktionen sind definiert. Jetzt müssen wir noch die Loops zur Ausführung der Datenabfrage und Erstellung eines ganzen Datenframes bauen. 

In [ ]:
path = 'MedReg_Data_50/'

file_list = os.listdir(path)

file = open(path + file_list, 'r')
text = file.read()
soup = BeautifulSoup(text, 'html.parser')

medreg_total = []

for html_file in tqdm(lst):
    
    file = open

Code-Elemente von anderen Beispielen:

In [ ]:
full_bäck_lst = []
for html_file in tqdm(lst):
    
    file = open('pages/' + html_file, 'r')
    text = file.read()
    h = BeautifulSoup(text, 'html.parser')
    
    full_bäck_lst = full_bäck_lst + get_bäcks(h)
    
#Einlesen und abspreichern:
df = pd.DataFrame(full_bäck_lst)
df.to_csv('bäckereien_list.csv')
    

#Und jetzt eben auf alle Files:

path = 'MedReg_Scraped_Data_20K/'
file_list = os.listdir(path)

medreg_final = []

for filename in file_list:    
    file = open(path + filename, 'r')
    text = file.read()
    h = BeautifulSoup(text, 'html.parser')

    medreg_final = medreg_final + extract(h)


## Projektvorstellung 10.12.2019 (Barny)

In [3]:
ls "MedReg_Data_50"

medreg_200.html  medreg_213.html  medreg_226.html  medreg_239.html
medreg_201.html  medreg_214.html  medreg_227.html  medreg_240.html
medreg_202.html  medreg_215.html  medreg_228.html  medreg_241.html
medreg_203.html  medreg_216.html  medreg_229.html  medreg_242.html
medreg_204.html  medreg_217.html  medreg_230.html  medreg_243.html
medreg_205.html  medreg_218.html  medreg_231.html  medreg_244.html
medreg_206.html  medreg_219.html  medreg_232.html  medreg_245.html
medreg_207.html  medreg_220.html  medreg_233.html  medreg_246.html
medreg_208.html  medreg_221.html  medreg_234.html  medreg_247.html
medreg_209.html  medreg_222.html  medreg_235.html  medreg_248.html
medreg_210.html  medreg_223.html  medreg_236.html  medreg_249.html
medreg_211.html  medreg_224.html  medreg_237.html
medreg_212.html  medreg_225.html  medreg_238.html


In [4]:
testliste = os.listdir("MedReg_Data_50")
path = "MedReg_Data_50/"

Ergebnis: sobald wir den Loop laufen lassen, gibt es Fehler, da gewisse HTML's leer sind oder bestimmte Tags ebenfalls keine Werte enthalten. 


Massnahme 1: So wenig formattieren wie möglich. Also bei BS4 den Befehl *.text* weglassen.


In [5]:
df = pd.DataFrame([])
for doc in testliste:
    print(doc)
    file = open(path + doc, 'r')   #file = open(path + doc, 'r')
    text = file.read()
    soup = BeautifulSoup(text, 'html.parser')

    name = soup.find('h3')
    nationalität = soup.find('p')
    geschlecht = get_gender(soup)
    #adresse = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
    beruf = soup.find('tbody')
    #jahr_diplom = soup.find('tbody').find('td').find_next('td')
    #land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td')
    #datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td')
    #weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td')
    #jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td')
    #land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td')
    #land_wbtitel = "NaN"
    minidict = {'Name': name,
               #'Adresse': adresse,
                'Geschlecht': geschlecht,
                'Beruf': beruf,
                #'Jahr Diplom': jahr_diplom,
                #'Land Diplom': land_diplom,
                #'Anerkennungsdatum': datum_anerkennung,
                #'Weiterbildung': weiterbildungstitel,
                #'Jahr Weiterbildung': jahr_wbtitel,
                #'Land Weiterbildung': land_wbtitel,
                'Nation': nationalität,
                'ID': doc
               }
    
    dftemp = pd.DataFrame([minidict])
    frames = [df, dftemp]
    df = pd.concat(frames)
    

medreg_224.html
medreg_232.html
medreg_249.html
medreg_208.html
medreg_228.html
medreg_245.html
medreg_212.html
medreg_204.html
medreg_205.html
medreg_213.html
medreg_244.html
medreg_229.html
medreg_209.html
medreg_248.html
medreg_233.html
medreg_225.html
medreg_238.html
medreg_243.html
medreg_214.html
medreg_202.html
medreg_222.html
medreg_234.html
medreg_218.html
medreg_219.html
medreg_235.html
medreg_223.html
medreg_203.html
medreg_215.html
medreg_242.html
medreg_239.html
medreg_200.html
medreg_216.html
medreg_241.html
medreg_236.html
medreg_220.html
medreg_221.html
medreg_237.html
medreg_240.html
medreg_217.html
medreg_201.html
medreg_230.html
medreg_226.html
medreg_206.html
medreg_210.html
medreg_247.html
medreg_246.html
medreg_211.html
medreg_207.html
medreg_227.html
medreg_231.html


In [6]:
df.head(5)

,Name,Geschlecht,Beruf,Nation,ID
0,"[Niederer, Margrit , []]",w,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...","[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_224.html
0,"[Sydler, Titus , []]",m,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...","[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_232.html
0,"[Boujon, Claude Eric , []]",m,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...","[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_249.html
0,"[Deplazes, Peter , []]",m,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...","[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_208.html
0,"[Schelling, Claude Pierre , []]",m,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...","[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_228.html


# Erkenntnisse aus Projektvorstellung (10.12.2019)

**Learning: in kleinen Schritten weitergehen ("Babysteps").**

1. Also: zuerst nur die wesentlichen Informationen einlesen.
2. Dann: sortieren, ausmisten. 
3. Grob auswerten.
4. Dann: bereinigen, formatieren.
5. Fein auswerten. 

Zuerst also mit diesen Daten arbeiten
ID, Name, Beruf, Weiterbildung (?), Nationalität

Über ID lassen sich später auch mehrere Datenframes zusammenführen. 



Zentral für eine erste Auswertung ist der Weiterbildungstitel. 


In [7]:
testliste = os.listdir("MedReg_Data_50")
path = "MedReg_Data_50/"

df = pd.DataFrame([])

for doc in testliste:
    
    print(doc)
    file = open(path + doc, 'r')
    text = file.read()
    soup = BeautifulSoup(text, 'html.parser')

    name = soup.find('h3')
    nationalität = soup.find('p')
    geschlecht = get_gender(soup)
    #adresse = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
    beruf = soup.find('tbody')
    #jahr_diplom = soup.find('tbody').find('td').find_next('td')
    #land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td')
    #datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td')
    weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td')
    #jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td')
    #land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td')
    #land_wbtitel = "NaN"
    minidict = {'Name': name,
               #'Adresse': adresse,
                'Geschlecht': geschlecht,
                'Beruf': beruf,
                #'Jahr Diplom': jahr_diplom,
                #'Land Diplom': land_diplom,
                #'Anerkennungsdatum': datum_anerkennung,
                'Weiterbildung': weiterbildungstitel,
                #'Jahr Weiterbildung': jahr_wbtitel,
                #'Land Weiterbildung': land_wbtitel,
                'Nation': nationalität,
                'ID': doc
               }
    
    dftemp = pd.DataFrame([minidict])
    frames = [df, dftemp]
    df = pd.concat(frames)
    

medreg_224.html
medreg_232.html
medreg_249.html
medreg_208.html
medreg_228.html
medreg_245.html
medreg_212.html
medreg_204.html
medreg_205.html


AttributeError: 'NoneType' object has no attribute 'find'

In [8]:
df.head(3)

,Name,Geschlecht,Beruf,Weiterbildung,Nation,ID
0,"[Niederer, Margrit , []]",w,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...",[Keine Angaben vorhanden],"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_224.html
0,"[Sydler, Titus , []]",m,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...",[Keine Angaben vorhanden],"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_232.html
0,"[Boujon, Claude Eric , []]",m,"[\n, [\n, [Tierärztin/Tierarzt], \n, [1984], \...",[Keine Angaben vorhanden],"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_249.html


Das Dokument *medreg_205.html* ist ein leerer Datensatz, der als Fehlermeldung zurückkam. Offenbar hängt sich BSoup da auf. 

Es ist zu erwarten, dass es bei fast 150'000 Datensätzen noch zu mehr solchen Problemen kommt. 

Die Frage lautet: ist es möglich alle Dateien zu durchsuchen und nur jene Files zu verwenden, in denen Psychotherapie vorkommt?

https://realpython.com/working-with-files-in-python/


## Die Lösung: die Suche in MacOS 
Der einfachste Weg: die Suche mit MacOS nach Dateiinhalt *Psychiatrie und Psychotherapie*. Es dauert Sekundenbruchteile und 5207 Files sind gefunden und in einen neuen Ordner *MedReg_Selection* kopiert.

Wir starten nun einen neuen Versuch, das Parsing durchzuführen mit den selektierten Files.

In [12]:
testliste = os.listdir("MedReg_Selection")
path = "MedReg_Selection/"

df = pd.DataFrame([])

for doc in testliste:
    
    print(doc)
    file = open(path + doc, 'r')
    text = file.read()
    soup = BeautifulSoup(text, 'html.parser')

    name = soup.find('h3')
    nationalität = soup.find('p')
    geschlecht = get_gender(soup)
    beruf = soup.find('tbody').find('td').text
    jahr_diplom = soup.find('tbody').find('td').find_next('td')
    land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td')
    datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td')
    weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td')
    jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td')
    land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td')
    #land_wbtitel = "NaN"
    minidict = {'Name': name,
                #'Adresse': adresse,
                'Geschlecht': geschlecht,
                'Beruf': beruf,
                'Jahr Diplom': jahr_diplom,
                'Land Diplom': land_diplom,
                'Anerkennungsdatum': datum_anerkennung,
                'Weiterbildung': weiterbildungstitel,
                'Jahr Weiterbildung': jahr_wbtitel,
                'Land Weiterbildung': land_wbtitel,
                'Nation': nationalität,
                'ID': doc
               }
    
    dftemp = pd.DataFrame([minidict])
    frames = [df, dftemp]
    df = pd.concat(frames)

medreg_27259.html
medreg_55166.html
medreg_68248.html
medreg_15778.html
medreg_34926.html
medreg_14169.html
medreg_35321.html
medreg_45023.html
medreg_40671.html
medreg_72700.html
medreg_80569.html
medreg_41060.html
medreg_67502.html
medreg_40558.html
medreg_31948.html
medreg_72429.html
medreg_44848.html
medreg_64457.html
medreg_26624.html
medreg_41349.html
medreg_43724.html
medreg_4158.html
medreg_71655.html
medreg_58538.html
medreg_33133.html
medreg_74142.html
medreg_71710.html
medreg_93083.html
medreg_33099.html
medreg_32722.html
medreg_75753.html
medreg_72096.html
medreg_64512.html
medreg_42070.html
medreg_16103.html
medreg_101473.html
medreg_94068.html
medreg_101920.html
medreg_4363.html
medreg_87214.html
medreg_24834.html
medreg_5964.html
medreg_12005.html
medreg_9414.html
medreg_28544.html
medreg_55074.html
medreg_24137.html
medreg_16046.html
medreg_40626.html
medreg_38544.html
medreg_30174.html
medreg_65211.html
medreg_43289.html
medreg_42861.html
medreg_18371.html
medreg_65641

medreg_93830.html
medreg_76866.html
medreg_2583.html
medreg_87873.html
medreg_37490.html
medreg_13435.html
medreg_15918.html
medreg_10130.html
medreg_90965.html
medreg_36452.html
medreg_15332.html
medreg_18150.html
medreg_33200.html
medreg_79443.html
medreg_14870.html
medreg_11721.html
medreg_21397.html
medreg_23200.html
medreg_7152.html
medreg_23715.html
medreg_45039.html
medreg_98513.html
medreg_77518.html
medreg_18916.html
medreg_5883.html
medreg_65175.html
medreg_43047.html
medreg_33345.html
medreg_9009.html
medreg_93148.html
medreg_33152.html
medreg_35985.html
medreg_44595.html
medreg_33502.html
medreg_65698.html
medreg_26350.html
medreg_78100.html
medreg_78550.html
medreg_19043.html
medreg_37404.html
medreg_11166.html
medreg_98211.html
medreg_37054.html
medreg_24678.html
medreg_23914.html
medreg_22606.html
medreg_13858.html
medreg_17118.html
medreg_58559.html
medreg_1911.html
medreg_72331.html
medreg_13788.html
medreg_3886.html
medreg_2794.html
medreg_30512.html
medreg_72761.html

medreg_784.html
medreg_93466.html
medreg_25052.html
medreg_36691.html
medreg_1615.html
medreg_76125.html
medreg_4417.html
medreg_21857.html
medreg_55645.html
medreg_31504.html
medreg_25117.html
medreg_2890.html
medreg_37829.html
medreg_32778.html
medreg_30745.html
medreg_58974.html
medreg_35547.html
medreg_19078.html
medreg_5656.html
medreg_5482.html
medreg_34881.html
medreg_41128.html
medreg_68245.html
medreg_4293.html
medreg_36445.html
medreg_3106.html
medreg_3413.html
medreg_34497.html
medreg_14888.html
medreg_2202.html
medreg_78300.html
medreg_35286.html
medreg_71988.html
medreg_89315.html
medreg_69141.html
medreg_87022.html
medreg_15149.html
medreg_43729.html
medreg_2982.html
medreg_96633.html
medreg_82220.html
medreg_41344.html
medreg_83962.html
medreg_13567.html
medreg_78183.html
medreg_8526.html
medreg_72424.html
medreg_44845.html
medreg_97472.html
medreg_7329.html
medreg_26296.html
medreg_33997.html
medreg_37487.html
medreg_2594.html
medreg_19185.html
medreg_30712.html
medreg_

medreg_38734.html
medreg_7380.html
medreg_92323.html
medreg_30754.html
medreg_86360.html
medreg_72527.html
medreg_89095.html
medreg_87988.html
medreg_27642.html
medreg_65431.html
medreg_7800.html
medreg_33601.html
medreg_15363.html
medreg_98057.html
medreg_43850.html
medreg_5094.html
medreg_82909.html
medreg_7046.html
medreg_64365.html
medreg_29349.html
medreg_23744.html
medreg_35690.html
medreg_36116.html
medreg_44229.html
medreg_59948.html
medreg_41181.html
medreg_44679.html
medreg_41355.html
medreg_1346.html
medreg_73674.html
medreg_71649.html
medreg_76925.html
medreg_8167.html
medreg_14749.html
medreg_4847.html
medreg_40401.html
medreg_44911.html
medreg_4902.html
medreg_98779.html
medreg_8022.html
medreg_12388.html
medreg_17470.html
medreg_28333.html
medreg_105752.html
medreg_12672.html
medreg_40952.html
medreg_33085.html
medreg_35852.html
medreg_43387.html
medreg_23986.html
medreg_84189.html
medreg_16148.html
medreg_5493.html
medreg_6315.html
medreg_15334.html
medreg_41569.html
me

medreg_36823.html
medreg_14544.html
medreg_42774.html
medreg_26065.html
medreg_22026.html
medreg_94412.html
medreg_36435.html
medreg_3176.html
medreg_98962.html
medreg_45398.html
medreg_8413.html
medreg_30762.html
medreg_4963.html
medreg_14397.html
medreg_31489.html
medreg_44073.html
medreg_93154.html
medreg_67343.html
medreg_11879.html
medreg_73245.html
medreg_76047.html
medreg_79218.html
medreg_45727.html
medreg_30627.html
medreg_77656.html
medreg_42518.html
medreg_30277.html
medreg_42148.html
medreg_40175.html
medreg_5220.html
medreg_45399.html
medreg_30763.html
medreg_66103.html
medreg_38353.html
medreg_45663.html
medreg_4962.html
medreg_85828.html
medreg_54971.html
medreg_28353.html
medreg_87546.html
medreg_108550.html
medreg_91538.html
medreg_73244.html
medreg_43308.html
medreg_93440.html
medreg_40877.html
medreg_66046.html
medreg_93913.html
medreg_40524.html
medreg_5735.html
medreg_7708.html
medreg_18423.html
medreg_65113.html
medreg_33289.html
medreg_23289.html
medreg_78721.htm

...

medreg_46827.html
medreg_13168.html
medreg_69108.html
medreg_101175.html
medreg_11790.html
medreg_83301.html
medreg_35735.html
medreg_73505.html
medreg_77815.html
medreg_6208.html
medreg_82140.html
medreg_5977.html
medreg_55437.html
medreg_93614.html
medreg_15696.html
medreg_34061.html
medreg_31799.html
medreg_72251.html
medreg_16110.html
medreg_66838.html
medreg_65717.html
medreg_73293.html
medreg_33527.html
medreg_64043.html
medreg_14711.html
medreg_38384.html
medreg_2861.html
medreg_9684.html
medreg_71611.html
medreg_94169.html
medreg_30999.html
medreg_41423.html
medreg_14896.html
medreg_86154.html
medreg_40398.html
medreg_45525.html
medreg_43808.html
medreg_67454.html
medreg_101964.html
medreg_17756.html
medreg_10093.html
medreg_5865.html
medreg_4327.html
medreg_93839.html
medreg_78488.html
medreg_23120.html
medreg_33120.html
medreg_36322.html
medreg_12387.html
medreg_43222.html
medreg_35918.html
medreg_40818.html
medreg_27026.html
medreg_3374.html
medreg_76183.html
medreg_39038.ht

medreg_31078.html
medreg_7747.html
medreg_11564.html
medreg_8148.html
medreg_78447.html
medreg_37456.html
medreg_2115.html
medreg_90673.html
medreg_30669.html
medreg_31994.html
medreg_78152.html
medreg_68851.html
medreg_7252.html
medreg_34295.html
medreg_37840.html
medreg_71373.html
medreg_85534.html
medreg_36302.html
medreg_32438.html
medreg_19768.html
medreg_19292.html
medreg_20956.html
medreg_92588.html
medreg_6290.html
medreg_3092.html
medreg_17776.html
medreg_17326.html
medreg_109924.html
medreg_4612.html
medreg_41053.html
medreg_17633.html
medreg_55440.html
medreg_39761.html
medreg_22087.html
medreg_41403.html
medreg_34503.html
medreg_59930.html
medreg_5003.html
medreg_110713.html
medreg_45440.html
medreg_72699.html
medreg_109162.html
medreg_68902.html
medreg_35884.html
medreg_11821.html
medreg_88255.html
medreg_36251.html
medreg_17449.html
medreg_22707.html
medreg_74167.html
medreg_18246.html
medreg_33116.html
medreg_43644.html
medreg_34779.html
medreg_31081.html
medreg_54882.ht

medreg_7701.html
medreg_71620.html
medreg_54997.html
medreg_42843.html
medreg_74422.html
medreg_94226.html
medreg_31481.html
medreg_17419.html
medreg_44581.html
medreg_27555.html
medreg_40891.html
medreg_33146.html
medreg_744.html
medreg_102857.html
medreg_104080.html
medreg_69355.html
medreg_11467.html
medreg_16208.html
medreg_68817.html
medreg_32583.html
medreg_73975.html
medreg_45514.html
medreg_38024.html
medreg_71948.html
medreg_40716.html
medreg_2692.html
medreg_43090.html
medreg_41557.html
medreg_590.html
medreg_31205.html
medreg_9174.html
medreg_14848.html
medreg_59921.html
medreg_98878.html
medreg_5442.html
medreg_77170.html
medreg_16463.html
medreg_83737.html
medreg_92126.html
medreg_59188.html
medreg_29770.html
medreg_92576.html
medreg_55778.html
medreg_81970.html
medreg_15536.html
medreg_20382.html
medreg_31069.html
medreg_85460.html
medreg_33404.html
medreg_29459.html
medreg_69617.html
medreg_90398.html
medreg_45778.html
medreg_30678.html
medreg_35180.html
medreg_33957.htm

medreg_7297.html
medreg_15274.html
medreg_58608.html
medreg_43044.html
medreg_15624.html
medreg_58258.html
medreg_3407.html
medreg_33346.html
medreg_23716.html
medreg_11667.html
medreg_36847.html
medreg_110739.html
medreg_92058.html
medreg_11722.html
medreg_42340.html
medreg_65033.html
medreg_88455.html
medreg_74622.html
medreg_26902.html
medreg_65463.html
medreg_85637.html
medreg_36001.html
medreg_3755.html
medreg_64966.html
medreg_65674.html
medreg_55368.html
medreg_40869.html
medreg_71637.html
medreg_21880.html
medreg_41281.html
medreg_9708.html
medreg_108958.html
medreg_19555.html
medreg_31880.html
medreg_37057.html
medreg_90788.html
medreg_2514.html
medreg_78416.html
medreg_33852.html
medreg_45387.html
medreg_32310.html
medreg_85836.html
medreg_3610.html
medreg_33151.html
medreg_15433.html
medreg_37811.html
medreg_6042.html
medreg_69591.html
medreg_13334.html
medreg_97671.html
medreg_65418.html
medreg_4706.html
medreg_55554.html
medreg_17232.html
medreg_34552.html
medreg_92865.htm

medreg_36934.html
medreg_77038.html
medreg_30049.html
medreg_28880.html
medreg_3827.html
medreg_83690.html
medreg_2220.html
medreg_36871.html
medreg_89337.html
medreg_80815.html
medreg_82081.html
medreg_3431.html
medreg_15612.html
medreg_39287.html
medreg_68288.html
medreg_25924.html
medreg_72056.html
medreg_4874.html
medreg_44867.html
medreg_19518.html
medreg_90395.html
medreg_33409.html
medreg_35924.html
medreg_73217.html
medreg_82652.html
medreg_41366.html
medreg_17013.html
medreg_21988.html
medreg_93556.html
medreg_81838.html
medreg_45260.html
medreg_29511.html
medreg_45630.html
medreg_72797.html
medreg_9987.html
medreg_38584.html
medreg_83278.html
medreg_91615.html
medreg_15350.html
medreg_12495.html
medreg_6721.html
medreg_82469.html
medreg_43475.html
medreg_74356.html
medreg_36125.html
medreg_58239.html
medreg_17382.html
medreg_14404.html
medreg_69134.html
medreg_16439.html
medreg_55958.html
medreg_83297.html
medreg_42634.html
medreg_37734.html
medreg_2277.html
medreg_34231.html

In [11]:
#Funktioniert nur im Ansatz. 

df.head(5)

,Name,Adresse,Geschlecht,Beruf,Jahr Diplom,Land Diplom,Anerkennungsdatum,Weiterbildung,Jahr Weiterbildung,Land Weiterbildung,Nation,ID
0,"[Withalm, Gerhard , []]","Zürcherstrasse 4,5620 Bremgarten AG,AG",m,"[\n, [\n, [Ärztin/Arzt], \n, [2006], \n, [Öste...",[2006],[Österreich],"[Anerkennung, [ ], \n (28.0...",[Psychiatrie und Psychotherapie],[2013],[Schweiz],"[Nationalität: Österreich (AT), [], Sprachkenn...",medreg_27259.html
0,"[Schneider, Werner , []]","Hauptstr. 54,4132 Muttenz,BL",m,"[\n, [\n, [Ärztin/Arzt], \n, [1982], \n, [Schw...",[1982],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1989],[Schweiz],"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_55166.html
0,"[Klein, Katrin Stephanie , []]","Untere Zollgasse 99,3063 Ittigen,BE",w,"[\n, [\n, [Ärztin/Arzt], \n, [2000], \n, [Deut...",[2000],[Deutschland],"[Anerkennung, [ ], \n (26.0...",[Psychiatrie und Psychotherapie],[2008],[Deutschland],"[Nationalität: Deutschland (DE), [], Sprachken...",medreg_68248.html
0,"[Leinfelder, Holger Ralf , []]","Austr. 79,4051 Basel,BS",m,"[\n, [\n, [Ärztin/Arzt], \n, [1994], \n, [Deut...",[1994],[Deutschland],"[Anerkennung, [ ], \n (23.0...",[Psychiatrie und Psychotherapie],[2002],[Schweiz],"[Nationalität: Deutschland (DE), [], Sprachken...",medreg_15778.html
0,"[Jobin, Peter , []]","Standweg 20,2540 Grenchen,SO",m,"[\n, [\n, [Ärztin/Arzt], \n, [1982], \n, [Schw...",[1982],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1990],[Schweiz],"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_34926.html


Aber eben, bei *medreg_67502.html* ist Schluss. 
Wäre ja zu schön, dass das einfach durchläuft. Fehlende Adresse.

Der Eintrag ist von einem Arzt, der zwar im Register ist, aber keine aktive Bewilligung hat und den Beruf wohl nicht ausübt. Es sind rund 1000 solcher Fälle zu erwarten. 

<?xml version="1.0" encoding="utf-8"?>
<div>
  <div style="min-height:51px">
    <h3>Harig, Jens <img class="geschlecht-icon" src="../../Images/mars-solid.png" /></h3>
    <div class="favorit detail"> </div>
    <div style="clear:left" />
  </div>
  <p>Nationalität: Deutschland (DE)<br />Sprachkenntnisse: Deutsch<br />GLN: 7601000742117<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) "> </a><br />UID: <a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gründen identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge für die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen geführt."> </a></p>
</div>
<div class="right">
  <div id="noresult_overlay">
    <div id="noresult_message" />
  </div>
  <div id="map" />
</div>
<div class="left">
  <ul class="detail-navigation">
    <li id="1">Ärztin/Arzt</li>
  </ul>
  <div id="detail_1" class="ac-detail-data">
    <div>
      <table class="detail-table">
        <thead>
          <tr>
            <td class="head">Beruf</td>
            <td class="head">Erteilungsjahr</td>
            <td class="head">Erteilungsland</td>
            <td class="head">Typ</td>
          </tr>
        </thead>
        <tbody>
          <tr>
            <td>Ärztin/Arzt</td>
            <td>2004</td>
            <td>Deutschland</td>
            <td>Anerkennung<a class="info" title="Datum der Anerkennung des ausländischen Titels durch die Schweiz: Datum nach 1.6.2002 Anerkennung für EU/EFTA-Bürger durch die Bundesbehörden. Datum vor 1.6.2002 Bestätigung durch den Kanton."> </a>
                    (08.12.2010)
                  </td>
          </tr>
          <tr>
            <th class="in-table-head">Weiterbildungstitel</th>
            <th />
            <th />
            <th />
          </tr>
          <tr>
            <td>Psychiatrie und Psychotherapie</td>
            <td>2014</td>
            <td>Schweiz</td>
            <td>Eidgenössischer Weiterbildungstitel</td>
          </tr>
          <tr>
            <td>Praktische Ärztin _ Praktischer Arzt</td>
            <td>2014</td>
            <td>Schweiz</td>
            <td>Eidgenössischer Weiterbildungstitel</td>
          </tr>
          <tr>
            <th class="in-table-head">Weitere Qualifikationen (privatrechtliche Weiterbildung)</th>
            <th />
            <th />
            <th />
          </tr>
          <tr>
            <td colspan="4">Keine Angaben vorhanden</td>
          </tr>
        </tbody>
      </table>
    </div>
    <hr />
    <h4>Status der Berufsausübungsbewilligung<a class="info" title="Erteilt, aktiv = gültige Berufsausübungsbewilligung für diesen Kanton, aktiv / Erteilt, inaktiv = gültige Berufsausübungsbewilligung für diesen Kanton, inaktiv / Keine Bewilligung, Bewilligung entzogen = Berufsausübungsbewilligung in diesem Kanton entzogen / Keine Bewilligung, Bewilligung verweigert = Berufsausübungsbewilligung in diesem Kanton verweigert / Keine Bewilligung = keine Bewilligung beantragt oder Berufsausübungsverbot"> </a></h4>Keine Bewilligung<h4>Direktabgabe von Arzneimitteln gemäss kant. Bestimmungen (Selbstdispensation)<a class="info" title="Keine Selbstdispensation: Darf nur in Sonderfällen (z.B. Notfällen) Medikamente abgeben / Keine Angaben vorhanden: keine kantonalen Daten vorhanden /  Bewilligung erteilt für Adresse(n): Darf Medikamente abgeben, d.h. ist zum Führen einer Privatapotheke berechtigt. / Es gelten die kantonalen Bestimmungen."> </a></h4>Keine Angaben vorhanden<h4>Bezug von Betäubungsmitteln<a class="info" title="Dieses Feld dient zur Information der Pharmagrossisten für die Belieferung "> </a></h4>Keine Angaben vorhanden<hr /></div>
</div>
<div class="clearRight"></div>

<script type="text/javascript">
	$(document).ready(function() {
		try {
			var _json_test = JSON;
		} catch (e) {
			document.createStyleSheet('/Content/ie8_compatview.css');
		}
	});
</script>



Ein Testdurchlauf ohne Adresse ergibt: es läuft durch. Allerdings möchte ich für meine Datenanalyse auch die Adresse verwenden. 

Mögliche Lösung: wir definieren eine Funktion, die die Adresse überpüft und sonst einen NaN-Wert ausgibt. 
https://stackoverflow.com/questions/35678699/how-do-i-ignore-empty-tags-with-beautiful-soup


Aber zuerst speichern wir mal die gefundenen Daten ab (ohne Adresse)



In [14]:
export_csv = df.to_csv (r'export_medreg_psych_2019_part.csv', index = None, header=True)

Hat geklappt. Die Daten sind aber nicht sehr sauber. Neben der Adressenproblematik sollten wir dann auch noch schauen, bei welchen Feldern wir direkt mit dem BS-Tag text arbeiten können, um die Einträge möglichst sauber zu überführen. 

In [37]:
def get_adress(soup):
    try:
        adress = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
    except:
        return "Kein Eintrag"

In [ ]:
doc_list = os.listdir("MedReg_Selection_100")
path = "MedReg_Selection_100/"

df = pd.DataFrame([])

for doc in doc_list:   
    print(doc) #zur Kontrolle, wie der Prozess durchläuft
    file = open(path + doc, 'r')
    text = file.read()
    soup = BeautifulSoup(text, 'html.parser')

    name = soup.find('h3').text.replace('\xa0','')
    nationalität = soup.find('p') 
    #hier verzichten wir auf das .text, mit den br-Tags lassen sich die Daten später einfache trennen.
    geschlecht = get_gender(soup) #die Funktion zum Auslesen des Geschlechts
    adresse = get_adress(soup)
    #beruf = soup.find('tbody').text #einfache Variante, liefert aber gescrambelte Daten.
    beruf = soup.find('tbody').find('td').text
    jahr_diplom = soup.find('tbody').find('td').find_next('td').text
    land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td').text
    datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td').text
    weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').text
    jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').text
    land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td').text
    
    minidict = {'Name': name,
                'Geschlecht': geschlecht,
                'Adresse': adresse,
                'Beruf': beruf,
                'Jahr Diplom': jahr_diplom,
                'Land Diplom': land_diplom,
                'Anerkennungsdatum': datum_anerkennung,
                'Weiterbildung': weiterbildungstitel,
                'Jahr Weiterbildung': jahr_wbtitel,
                'Land Weiterbildung': land_wbtitel,
                'Nationalität': nationalität,
                'ID': doc
               }
    
    dftemp = pd.DataFrame([minidict])
    frames = [df, dftemp]
    df = pd.concat(frames)

In [19]:
df.head(-5)

,Name,Adresse,Geschlecht,Beruf,Jahr Diplom,Land Diplom,Anerkennungsdatum,Weiterbildung,Jahr Weiterbildung,Land Weiterbildung,Nation,ID
0,"Hegland, Ingvar Nicolai Thomas",None,m,"[\n, [\n, [Ärztin/Arzt], \n, [1985], \n, [Schw...",[1985],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1997],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: De...,medreg_39969.html
0,"Koch, Urs",None,m,"[\n, [\n, [Ärztin/Arzt], \n, [1981], \n, [Schw...",[1981],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1990],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: De...,medreg_39894.html
0,"Fischer, Roman",None,m,"[\n, [\n, [Ärztin/Arzt], \n, [1980], \n, [Schw...",[1980],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1988],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: De...,medreg_38968.html
0,"Löblein, Christina",None,w,"[\n, [\n, [Ärztin/Arzt], \n, [1995], \n, [Deut...",[1995],[Deutschland],"[Anerkennung, [ ], \n (24.0...",[Psychiatrie und Psychotherapie],[2002],[Schweiz],Nationalität: Deutschland (DE)Sprachkenntnisse...,medreg_39441.html
0,"Platis, Anastasio",Kein Eintrag,m,"[\n, [\n, [Ärztin/Arzt], \n, [1978], \n, [Schw...",[1978],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1979],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: Fr...,medreg_40124.html
...,...,...,...,...,...,...,...,...,...,...,...,...
0,"Schmerek, Otto",Kein Eintrag,m,"[\n, [\n, [Ärztin/Arzt], \n, [1981], \n, [Schw...",[1981],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1987],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: De...,medreg_40243.html
0,"Merker, Jacques",None,m,"[\n, [\n, [Ärztin/Arzt], \n, [1975], \n, [Schw...",[1975],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1985],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: Fr...,medreg_39751.html
0,"Perrin-Zabilska, Nina",None,w,"[\n, [\n, [Ärztin/Arzt], \n, [1983], \n, [Schw...",[1983],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1990],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: De...,medreg_39028.html
0,"Kraan-Lang, Karel",None,m,"[\n, [\n, [Ärztin/Arzt], \n, [1979], \n, [Schw...",[1979],[Schweiz],[Eidgenössisches Diplom],[Psychiatrie und Psychotherapie],[1986],[Schweiz],Nationalität: Schweiz (CH)Sprachkenntnisse: De...,medreg_38951.html


Funktioniert, bis auf:
1. Die Adresse wird jetzt als None ausgegeben. 
2. Das Anerkennungsdatum enthält den Typ des Diploms
3. Nation enthält auch Sprachkenntnisse. Das sauber rauszukriegen wird nicht einfach (https://stackoverflow.com/questions/48216573/converting-br-to-p-p-in-python?noredirect=1&lq=1)

    

Test der Adressenfunktion, mit einem Datensatz. 

In [80]:
path = 'MedReg_Scraped_Data_20K/'
file_list = os.listdir(path)

file = open(path + file_list[4805], 'r')
text = file.read()
soup = BeautifulSoup(text, 'html.parser')

In [83]:
def get_adress(soup):
    try:
        soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
    except:
        return "Kein Eintrag"

In [85]:
adresse = get_adress(soup)
print(adresse)

None


In [91]:
#Also zuerst mal für sich alleine:
adresse = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
print(adresse)

Dufourstr. 169,8008 Zürich,ZH


Geht wunderbar. Wieso nicht als Funktion?

In [109]:
def get_adress(soup):
    try: 
        soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
    except:
        return "Kein Eintrag"

In [107]:
get_adress
adresse2

In [ ]:
adresse2 = get_adress(soup)
adresse2

Kein Output. In der Funktion fehlt möglicherweise ein return, deshalb gibt es einen «None» value, wenn ich die Funktion über 100 Files laufen lasse. Korrekt angezeigt mit «Kein Eintrag» werden jeweils nur die Datensätze, die keine Adresse enthalten. 


In [ ]:
def get_adress(soup):
    try: 
        adresse3 = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
        return adresse3
    except:
        return "Kein Eintrag"

In [111]:
adresse3 = get_adress(soup)
adresse3

'Dufourstr. 169,8008 Zürich,ZH'

Aha, so scheint es zu klappen. 

Jetzt noch zum Feld «Anerkennungsdatum»:


In [59]:
datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td').text
datum_anerkennung

'Anerkennung\xa0\n                    (13.08.2002)\n                  '

Es ist kein Problem. Bei Schweizer Diplomen kommt einfach Schweiz, bei anerkannten Diplomen sind die richtigen Werte enthalten. 
Also sollten wir soweit sein.

## Code für das Parsing

Die benötigten Bibliotheken:

In [62]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import re
import datetime
from tqdm import tqdm

Die zwei Spezialauslese-Funktionen.

In [112]:
def get_gender(soup):
    try:
        img_source = soup.find('h3').find("img")["src"]
        if "venus" in img_source:
            return "w"
        else:
            return "m"
    except:
        return "unbekannt"

def get_adress(soup):
    try: 
        adresse = soup.find_all("ol")[-1].find_all("li", {"class": "address-item"})[0]["title"]
        return adresse
    except:
        return "Kein Eintrag"

In [113]:
doc_list = os.listdir("MedReg_Selection_100")
path = "MedReg_Selection_100/"

df = pd.DataFrame([])

for doc in doc_list:   
    print(doc) #zur Kontrolle, wie der Prozess durchläuft
    file = open(path + doc, 'r')
    text = file.read()
    soup = BeautifulSoup(text, 'html.parser')
    
    name = soup.find('h3').text.replace('\xa0','')
    nationalität = soup.find('p') 
    #hier verzichten wir auf das .text, mit den br-Tags lassen sich die Daten später einfache trennen.
    geschlecht = get_gender(soup) #die Funktion zum Auslesen des Geschlechts
    adresse = get_adress(soup)
    beruf = soup.find('tbody').find('td').text
    jahr_diplom = soup.find('tbody').find('td').find_next('td').text
    land_diplom = soup.find('tbody').find('td').find_next('td').find_next('td').text
    datum_anerkennung = soup.find('tbody').find('td').find_next('td').find_next('td').find_next('td').text
    weiterbildungstitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').text
    jahr_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').text
    land_wbtitel = soup.find('tbody').find('tr').find_next('tr').find_next('tr').find('td').find_next('td').find_next('td').text
    
    minidict = {'Name': name,
                'Geschlecht': geschlecht,
                'Adresse': adresse,
                'Beruf': beruf,
                'Jahr Diplom': jahr_diplom,
                'Land Diplom': land_diplom,
                'Anerkennungsdatum': datum_anerkennung,
                'Weiterbildung': weiterbildungstitel,
                'Jahr Weiterbildung': jahr_wbtitel,
                'Land Weiterbildung': land_wbtitel,
                'Nationalität': nationalität,
                'ID': doc
               }
    
    dftemp = pd.DataFrame([minidict])
    frames = [df, dftemp]
    df = pd.concat(frames)

medreg_39969.html
medreg_39894.html
medreg_38968.html
medreg_39441.html
medreg_40124.html
medreg_39729.html
medreg_39436.html
medreg_39733.html
medreg_39288.html
medreg_39772.html
medreg_38837.html
medreg_39698.html
medreg_40001.html
medreg_40037.html
medreg_39456.html
medreg_38882.html
medreg_39440.html
medreg_38969.html
medreg_38912.html
medreg_38953.html
medreg_39616.html
medreg_39975.html
medreg_39430.html
medreg_38734.html
medreg_40401.html
medreg_39918.html
medreg_40092.html
medreg_38846.html
medreg_38981.html
medreg_38755.html
medreg_40088.html
medreg_38997.html
medreg_40175.html
medreg_39145.html
medreg_38754.html
medreg_40027.html
medreg_38851.html
medreg_40251.html
medreg_39364.html
medreg_39036.html
medreg_40400.html
medreg_39718.html
medreg_39962.html
medreg_39431.html
medreg_40380.html
medreg_39759.html
medreg_40116.html
medreg_39035.html
medreg_40157.html
medreg_40415.html
medreg_40073.html
medreg_39039.html
medreg_39890.html
medreg_39097.html
medreg_38957.html
medreg_403

In [114]:
df.head(5)

,Name,Geschlecht,Adresse,Beruf,Jahr Diplom,Land Diplom,Anerkennungsdatum,Weiterbildung,Jahr Weiterbildung,Land Weiterbildung,Nationalität,ID
0,"Hegland, Ingvar Nicolai Thomas",m,"Buchserstr. 7,8157 Dielsdorf,ZH",Ärztin/Arzt,1985,Schweiz,Eidgenössisches Diplom,Psychiatrie und Psychotherapie,1997,Schweiz,"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_39969.html
0,"Koch, Urs",m,"Via C. Maraini 13B,6900 LUGANO,TI",Ärztin/Arzt,1981,Schweiz,Eidgenössisches Diplom,Psychiatrie und Psychotherapie,1990,Schweiz,"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_39894.html
0,"Fischer, Roman",m,"Inselquai 10,6005 Luzern,LU",Ärztin/Arzt,1980,Schweiz,Eidgenössisches Diplom,Psychiatrie und Psychotherapie,1988,Schweiz,"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_38968.html
0,"Löblein, Christina",w,"Alte Landstr. 113,8707 Uetikon a.S.,ZH",Ärztin/Arzt,1995,Deutschland,Anerkennung \n (24.07.2002)...,Psychiatrie und Psychotherapie,2002,Schweiz,"[Nationalität: Deutschland (DE), [], Sprachken...",medreg_39441.html
0,"Platis, Anastasio",m,Kein Eintrag,Ärztin/Arzt,1978,Schweiz,Eidgenössisches Diplom,Psychiatrie und Psychotherapie,1979,Schweiz,"[Nationalität: Schweiz (CH), [], Sprachkenntni...",medreg_40124.html


Unser Code ist bereit für die 5000 gescrapten Psychiater-Files - im Notebook MedReg Parsing 03